In [1]:
from ramanujan.cmf import known_cmfs
from ramanujan.cmf import CMF
from ramanujan import Matrix

import mpmath
import sympy
from fractions import Fraction
from sympy.abc import x, y

from IPython.display import display, Math

# Use high percission
mpmath.mp.dps = 20_000

def pslq(limit, constant, constant_latex):
    coefs = mpmath.pslq([1, constant, -limit, -limit*constant], tol=1e-30)
    # create the latex that represents the limit
    c0 = '' if coefs[0] == 0 else str(coefs[0])
    c2 = '' if coefs[2] == 0 else str(coefs[2])
    
    if coefs[1] == 0:
        c1 = ''
    elif coefs[1] > 0:
        c1 = f'{coefs[1]}{constant_latex}'
        c1 = f'+ {c1}' if coefs[0] != 0 else c1
    else:
        c1 = f'{coefs[1]}{constant_latex}'
        
    if coefs[3] == 0:
        c3 = ''
    elif coefs[3] > 0:
        c3 = f'{coefs[3]}{constant_latex}'
        c3 = f'+ {c1}' if coefs[2] != 0 else c3
    else:
        c3 = f'{coefs[3]}{constant_latex}'
        
    
    expr_latex = f'\\frac{{ {c0} {c1} }}{{ {c2} {c3} }}'
    return coefs, expr_latex

# Connection between constants found by CMFs

Conservative matrix fields generate infinitely many formulas for fundamental mathematical constants. In most cases, these formulas correspond to a single constant. Sometimes, very simple and intuitive changes can be applied to the CMF, such as altering the initial configuration, leading to new formulas that converge to a different constant.

Both constants can be expressed with _nearly identical_ conservative matrix fields, which means that _nearly identical_ formulas lead to different constants. This observation implies that the constants are connected in some manner.

In this document we will explore such connections by applying various manipulations to the CMFs we have discovered.

## Rational indents
The _initial location_ of a trajectory on a conservative matrix field, can be manipulated by mutliplying by the matrices that correspond to the path that connects the original initial location, and the desired one. 
For example the trajectory the progresses on the $x=y$ diagonal is usually initialied on $(1,1)$

$$ V_{\mathrm{traj-limit}} = M_x(1,1) M_y(2,1) M_x(2,2) M_y(3,2) \dots M_x(n,n) M_y(n+1,n) \dots \to L $$ 
Where $L$ is the limit of the trajectory, defined as $V_{0,1}/V_{1,1}$.

The same limit, up to a Mobius transform, can be attained by using a different _integer_ initial location. For example, starting from $(3,1)$:
$$ M_x^{-1}(3,1) M_x^{-1}(2,1) V_{\mathrm{traj-limit}} \to M_x^{-1}(3,1) M_x^{-1}(2,1) L $$ 

However, If we choose a _rational_ initial location, we cannot express the path that connects both trajectories using a Mobius transform, which raises the question - will the new limit still be an expression of the old constant? Surprisingly, the answer is no.

In [2]:
def compare_limits_after_indent(cmf, indent, origianl_const, original_const_latex,
                                indent_const, indent_const_latex, n_iterations=1000):
    # We check the limit on the x=y diagonal. 
    # Usually, it provides the fastest convergence.
    original_limit = cmf.limit(
        trajectory={x:1, y:1},
        iterations=n_iterations,
        start={x:1, y:1})

    # The new matrix field is initiated at a location that differes by a rational value
    indented_limit = cmf.limit(
        trajectory={x:1, y:1},
        iterations=n_iterations,
        start={x:1 + indent[0], y:1+indent[1]})
    
    # We use PSLQ to identify a closed form of the limit constants
    print('Starting at x=y=1:')
    _, expr_latex = pslq(original_limit, origianl_const, original_const_latex)
    display(Math(f'Limit={mpmath.nstr(original_limit,20)} = {expr_latex}'))

    print(f'Starting at x=1+{indent[0]}, y=1+{indent[1]}:')
    _, expr_latex = pslq(indented_limit, indent_const, indent_const_latex)
    display(Math(f'Limit={mpmath.nstr(indented_limit,20)} = {expr_latex}'))

## Connection between $\pi$ and $\sqrt{2}$

In [ ]:
pi_cmf = known_cmfs.pi()
compare_limits_after_indent(
    cmf=pi_cmf,
    indent=(Fraction(0,2), Fraction(1,2)),
    origianl_const=mpmath.pi, 
    original_const_latex='\\pi',
    indent_const=mpmath.sqrt(2), 
    indent_const_latex='\\sqrt{2}')

## Connection between Catalan's constant $G$ and $\zeta(2)$

In [ ]:
zeta2_cmf = known_cmfs.zeta2()
compare_limits_after_indent(
    cmf=zeta2_cmf,
    indent=(0,Fraction(1,2)),
    origianl_const=mpmath.zeta(2), 
    original_const_latex='\\zeta(2)',
    indent_const=mpmath.catalan, 
    indent_const_latex='G')

## Connection between $\frac{\pi^3}{\sqrt{3}}$ and $\zeta(3)$

In [ ]:
zeta3_cmf = known_cmfs.zeta3()
# This value is infact polygamma(2, 4/3)
zeta3_pi_cube_mix = 54 - (4*mpmath.pi**3)/(3*mpmath.sqrt(3)) - 26*mpmath.zeta(3)
compare_limits_after_indent(
    cmf=zeta3_cmf,
    indent=(0,Fraction(1,3)),
    origianl_const=mpmath.zeta(3), 
    original_const_latex='\\zeta(3)',
    indent_const=zeta3_pi_cube_mix, 
    indent_const_latex='\\left(54-(4\\pi^3)/(3\\sqrt{3})-26\\zeta(3)\\right)')

## Connecting different constants by passing $|M_{x/y}|=0$

As seen above, to connect different trajectories by using the inverse of $M_x$ or $M_y$. In cases where determinant is zero, we once again cannont connect different trajectories, since we cannot invert $M_x$ or $M_y$. This gives a new motivation for looking for different constants in different areas of the conservative matrix field, based on the determinant of the matrices.

In [ ]:
e_cmf = known_cmfs.e()
display(Math(f'\\mathrm{{det}}(M_x) = {e_cmf.Mx.det()}'))
display(Math(f'\\mathrm{{det}}(M_y) = {e_cmf.My.det()}'))

In [ ]:
Mx_inv=Matrix([
    [ e_cmf.Mx[1,1], -e_cmf.Mx[0,1]],
    [-e_cmf.Mx[1,0],  e_cmf.Mx[0,0]]
])
My=e_cmf.My

In [ ]:
pcf = sympy.eye(2)
for i in range(1, 1000):
    # We want to go in the y=-x direction on the second quandrant.
    # The matrix that takes us a step back in the x direction, e.g. from (x_0, y_0) to (x_0-1, y_0), is the 
    # inverse of M_x(x_0-1, y_0)
    
    # step backwards in x, starting from (-i, i). After this operation, we are at (-i-1, i).
    pcf = pcf @ Mx_inv.subs({x:-i-1, y:i})
    # step upwards in y. This axis was not changed.
    pcf = pcf @ My.subs({x:-i-1, y:i})

quad_2_limit = mpmath.mpf(pcf[0,1]/pcf[1,1])

In [ ]:
gompertz_const = mpmath.mpf('0.59634736232319407434107849936927937607417786015254878157348491048232721911487441747043049709361276034423703474842862368981207829952905719661736922266589402431851351436829376329625477118797402524323020521178857378561772836523651378559486742535621813008120833')
_, gom_latex  = pslq(quad_2_limit, gompertz_const, '\\delta')

print('The limit on x=-y in the second quadrant is:')
display(Math(f'L = {mpmath.nstr(quad_2_limit, 20)} = {gom_latex}'))

print('Where 𝛿 is the Euler-Gompertz constant')

## Connection between constants by re-scaling the axes

Another simple modification we can apply to the conservative matrix field is a re-scaling of the axes, either by $x\to x/2$ or $y\to y+1/2$. This action preserves most of the properties of the original recurrence formula represented by a trajectory, creating a new, and very similar recurrence formula, that cannot be attained by simple Mobius transforms on a known trajectory.

## Connection between $\pi$ and $\ln(2)$

In [ ]:
Mx_rescaled = pi_cmf.Mx.subs({x:x/2, y:1})

pcf = sympy.eye(2)
for i in range(1, 1000):
    pcf = pcf @ Mx_rescaled.subs({x:i})
limit = mpmath.mpf(pcf[0,1]/pcf[1,1])

_, expr_latex = pslq(limit, mpmath.ln(2), '\\ln(2)')
display(Math(f'L_{{x \\to x/2}} = {mpmath.nstr(limit, 20)} = {expr_latex}'))